In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nw

import clean

In [9]:
g = nw.DiGraph()

In [10]:
df = clean.parse_json_to_df('data/raw.json')

In [23]:
df = df[~df['ingredients'].isnull()]
df['course'] = df.attributes.apply(lambda x: x['course'] if x.has_key('course') else [])
df['cuisine'] = df.attributes.apply(lambda x: x['cuisine'] if x.has_key('cuisine') else [])
df_cuisine = df[df['cuisine'].apply(lambda x: len(x)>0)]
df_course = df[df['course'].apply(lambda x: len(x)>0)]


In [26]:
df_desserts = df[df['course'].apply(lambda x: 'Desserts' in x)]

In [30]:
df_desserts

,attributes,flavors,id,imageUrlsBySize,ingredients,rating,recipeName,smallImageUrls,sourceDisplayName,totalTimeInSeconds,course,cuisine
4,{u'course': [u'Desserts']},"{u'sweet': 1.0, u'salty': 0.333333333333, u'so...",Dark-Chocolate-Ganache-Martha-Stewart-194113,{u'90': u'http://lh4.ggpht.com/ycGozZNUGhQ87Nn...,"[confectioners sugar, heavy cream, salt, bitte...",4,Dark-Chocolate Ganache,[http://lh3.ggpht.com/6ApI_mocLD9EX23MQ9yhgGkP...,Martha Stewart,4500,[Desserts],[]
5,{u'course': [u'Desserts']},"{u'sweet': 1.0, u'salty': 0.166666666667, u'bi...",The-Best-Chocolate-Sauce-513548,{u'90': u'http://lh5.ggpht.com/0pjOCRO5XQARRnh...,"[unsweetened cocoa powder, sugar, water, light...",5,The Best Chocolate Sauce,[http://lh4.ggpht.com/NFiOCL3xXYUhi-niorUBsv6w...,David Lebovitz,NaN,[Desserts],[]
7,{u'course': [u'Desserts']},"{u'sweet': 0.833333333333, u'salty': 0.3333333...",Raspberry-Mousse-Pie-Martha-Stewart-196892,{u'90': u'http://lh5.ggpht.com/R97feXMI8d_8DzH...,"[fresh lemon juice, graham cracker, heavy crea...",5,Raspberry Mousse Pie,[http://lh5.ggpht.com/N0bAXn048zZH8jzw_xbbYEx9...,Martha Stewart,2400,[Desserts],[]
22,"{u'course': [u'Desserts'], u'cuisine': [u'amer...","{u'sweet': 1.0, u'salty': 0.333333333333, u'so...",Crumb-Topping-Martha-Stewart-194242,{u'90': u'http://lh6.ggpht.com/QkDeFBqyJdma8f9...,"[light brown sugar, cinnamon, unsalted butter,...",3,Crumb Topping,[http://lh6.ggpht.com/QeV9UcNhilnnX1mhfyUcVoAw...,Martha Stewart,6300,[Desserts],[american]
23,{u'course': [u'Desserts']},"{u'sweet': 0.666666666667, u'salty': 0.8333333...",Foolproof-pie-dough-307222,{u'90': u'http://lh5.ggpht.com/84PUO8OnErfcqAO...,"[vodka, unbleached all purpose flour, table sa...",5,Foolproof Pie Dough,[http://lh4.ggpht.com/-P1ISX2jbU9N0x9enTtpC3fz...,Smitten Kitchen,3600,[Desserts],[]
36,{u'course': [u'Desserts']},"{u'sweet': 1.0, u'salty': 0.333333333333, u'bi...",Swiss-buttercream-307219,{u'90': u'http://lh6.ggpht.com/iWQcRjQf6Jssyov...,"[egg whites, large egg whites, unsalted butter...",5,Swiss Buttercream,[http://lh3.ggpht.com/s6WC2ZqsC_kkcFbIf-uTGS1J...,Smitten Kitchen,2100,[Desserts],[]
37,"{u'course': [u'Desserts'], u'cuisine': [u'amer...","{u'sweet': 1.0, u'salty': 0.333333333333, u'bi...",Peanut-Butter-Swirl-Bark-Martha-Stewart-196377,{u'90': u'http://lh5.ggpht.com/8pn7R0gEDYi97Mo...,"[creamy peanut butter, milk chocolate, bitters...",5,Peanut Butter Swirl Bark,[http://lh6.ggpht.com/UE8wwzqzD8unDXbHId12Q2-R...,Martha Stewart,4500,[Desserts],[american]
40,"{u'course': [u'Desserts'], u'cuisine': [u'amer...","{u'sweet': 0.166666666667, u'salty': 0.6666666...",Pie-Dough-Martha-Stewart_1,{u'90': u'http://lh3.ggpht.com/pcnVVfTbJfHcWbC...,"[ice water, unsalted butter, salt, sugar, all-...",3,Pie Dough,[http://lh6.ggpht.com/9q-jDy4W7LKCxDka6JOIUuV2...,Martha Stewart,4800,[Desserts],[american]
46,{u'course': [u'Desserts']},"{u'sweet': 0.333333333333, u'salty': 0.6666666...",Pate-a-Choux-Martha-Stewart,{u'90': u'http://lh6.ggpht.com/-RzHE929WY03wSS...,"[unsalted butter, large eggs, salt, sugar, all...",5,Pate a Choux,[http://lh6.ggpht.com/02-pU9GQdgs5RveU9ScCzqe4...,Martha Stewart,1800,[Desserts],[]
48,{u'course': [u'Desserts']},"{u'sweet': 0.833333333333, u'salty': 0.1666666...",Homemade-nutella-308338,{u'90': u'http://lh4.ggpht.com/IYwUASrg1bgh78j...,"[canola oil, powdered sugar, cocoa powder, kos...",5,Homemade Nutella,[http://lh4.ggpht.com/oQkz44dKhaeFZnoe8Dh0DQVe...,The Kitchn,1500,[Desserts],[]


In [24]:
df['course']

0               []
1               []
2         [Salads]
3               []
4       [Desserts]
5       [Desserts]
6               []
7       [Desserts]
8               []
9               []
10              []
11              []
12              []
13    [Appetizers]
14              []
...
707585              []
707586              []
707587              []
707588         [Soups]
707589              []
707590              []
707591         [Soups]
707592              []
707593              []
707594    [Appetizers]
707595              []
707596              []
707597        [Breads]
707598              []
707599      [Desserts]
Name: course, Length: 707507, dtype: object

# Build the Graph

For the graph, nodes are ingredients. This analysis looks at how ingredients are connected to each other. Edges are a result of recipes that use these ingredients. Here are some hypothesis hoping to answer:
- a logical connection between similar ingredients can be discovered by recipes that connect them together
- a logical indicator of the most important ingredients in any kitchen

In [46]:
# work with a smaller subset of data
df_subset = df.iloc[map(int,np.floor(np.random.uniform(0,len(df), 1000)))]

In [51]:
def make_ingredient_graph(df_set):
    g = nw.DiGraph()
    blacklist = []
    recipe_l = df_set['recipeName'].tolist()
    ingredient_l = df_set['ingredients'].tolist()
    for i in xrange(len(df_set)):
        ingredients = ingredient_l[i]
        recipe_name = recipe_l[i]


        # remove some super common ingredients
        for b in blacklist:
            if b in ingredients:
                ingredients.remove(b)
        # add the ingredient to the hash of nodes before adding edges
        for ing in ingredients:
            if not g.has_node(ing):
                g.add_node(ing)
        # add edges to each node now
        for ing in ingredients:
    #         nodes[ing]['recipes'].append(row['recipeName'])
            for ig in [ig for ig in ingredients if ig != ing]:
                if not g.edge[ing].has_key(ig):
                    g.add_weighted_edges_from([(ing, ig,0)])
                g.edge[ing][ig]['weight'] += 1
    return g

def page_rank_graph(g):
    ranks = nw.pagerank(g)
    df_ranks = pd.DataFrame(map(lambda x: dict(ingredient=x, prob=ranks[x]), ranks))
    return df_ranks.sort('prob', ascending=False)

In [52]:
g_desserts = make_ingredient_graph(df_desserts)
g_sampled = make_ingredient_graph(df_subset)

In [49]:
page_rank_graph(g_sampled)

,ingredient,prob
653,salt,0.037738
1247,onion,0.016226
238,sugar,0.014615
328,butter,0.014541
1109,eggs,0.013159
282,olive oil,0.013098
1733,pepper,0.012605
483,water,0.011933
1317,garlic cloves,0.011811
837,all-purpose flour,0.010179


In [53]:
page_rank_graph(g_desserts)

,ingredient,prob
2954,sugar,0.039935
488,salt,0.036846
3023,butter,0.031778
3603,eggs,0.028829
1142,all-purpose flour,0.023915
4838,vanilla extract,0.021668
1597,baking powder,0.019084
5133,baking soda,0.019016
1217,large eggs,0.015727
1779,unsalted butter,0.015551
